In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('tjornal.csv')

In [4]:
df.head()

,article_url,article_time,article_head,article_author,article_content
0,https://tjournal.ru/u/274224-anatoliy-tihvinsk...,11.11.2019 17:33:39 (Europe/Moscow),ИСКУССТВЕННЫЙ ИНТЕЛЛЕКТ,Анатолий Тихвинский,Люди создали Искусственный Интеллект и открыл...
1,https://tjournal.ru/internet/131681-iskusstven...,18.12.2019 00:15:06 (Europe/Moscow),Искусственный интеллект,Интернет и мемы,
2,https://tjournal.ru/tech/125621-iskusstvennyy-...,14.11.2019 04:41:58 (Europe/Moscow),Искусственный интеллект обучили копировать чуж...,Технологии,
3,https://tjournal.ru/news/116219-vkontakte-vper...,13.09.2019 12:38:13 (Europe/Moscow),«ВКонтакте» впервые откроет научную лаборатори...,Новости,Кураторами лаборатории станут разработчики ко...
4,https://tjournal.ru/tech/131933-kazhdyy-iz-nih...,23.12.2019 11:35:34 (Europe/Moscow),Каждый из них играл за человечество: история п...,Технологии,22 года развития ИИ в хронологии матчей – от ...


In [8]:
df.shape

(423, 6)

In [9]:
df.drop_duplicates(subset=['article_url']).shape

(396, 6)

In [6]:
df['article_length'] = df.article_content.apply(lambda x: len(x) if x is not np.nan else np.nan)

In [34]:
row[1]

article_url        https://tjournal.ru/internet/131681-iskusstven...
article_time                     18.12.2019 00:15:06 (Europe/Moscow)
article_head                                 Искусственный интеллект
article_author                                       Интернет и мемы
article_content       Не понял прекол, переведите на питон  В язы...
article_length                                                   421
Name: 1, dtype: object

In [7]:
for row in df.query('article_length < 500').itertuples(index=False):
    print(row[0], '\t', row[4])
    print('\n\nnext')

https://tjournal.ru/internet/131681-iskusstvennyy-intellekt 	  


next
https://tjournal.ru/tech/125621-iskusstvennyy-intellekt-obuchili-kopirovat-chuzhoy-golos-za-5-sekund-vhodnyh-dannyh 	  


next
https://tjournal.ru/u/264054-rushan-syurmakov/130812-daydzhest-novostey-mashinnogo-obucheniya-i-iskusstvennogo-intellekta-za-noyabr 	  Привет, я Рушан. И я делаю « Отфильтровав для вас множество источников и подписок, собрал все наиболее значимые новости из мира будущих технологий, роботов, машинного обучения и искусственного интеллекта за ноябрь. Для тех, кто не читал, — На этом наш короткий ноябрьский дайджест подошёл к концу. Не забудьте поделиться статьёй с друзьями или просто с теми, кому интересны такие новости. Не пропускать свежие статьи вам поможет Всем знаний! 


next
https://tjournal.ru/tech/85066-iskusstvennyy-intellekt-napisal-scenariy-dlya-reklamy-lexus-es-kotoruyu-snyal-oskaronosnyy-rezhisser 	  Компания хотела показать, что человек и машина могут гармонично сотрудничать. Lexu